# Mecanismos de atención y transformadores

Una de las principales desventajas de las redes recurrentes es que todas las palabras en una secuencia tienen el mismo impacto en el resultado. Esto provoca un rendimiento subóptimo en los modelos estándar de codificador-decodificador LSTM para tareas de secuencia a secuencia, como el Reconocimiento de Entidades Nombradas y la Traducción Automática. En realidad, palabras específicas en la secuencia de entrada suelen tener más impacto en las salidas secuenciales que otras.

Consideremos un modelo de secuencia a secuencia, como la traducción automática. Este se implementa mediante dos redes recurrentes, donde una red (**codificador**) colapsa la secuencia de entrada en un estado oculto, y otra (**decodificador**) desenvuelve este estado oculto en el resultado traducido. El problema con este enfoque es que el estado final de la red tiene dificultades para recordar el inicio de una oración, lo que provoca una calidad deficiente del modelo en oraciones largas.

Los **mecanismos de atención** proporcionan un medio para ponderar el impacto contextual de cada vector de entrada en cada predicción de salida de la RNN. Esto se implementa creando atajos entre los estados intermedios de la RNN de entrada y la RNN de salida. De esta manera, al generar el símbolo de salida $y_t$, tomaremos en cuenta todos los estados ocultos de entrada $h_i$, con diferentes coeficientes de peso $\alpha_{t,i}$. 

![Imagen que muestra un modelo codificador/decodificador con una capa de atención aditiva](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*El modelo codificador-decodificador con mecanismo de atención aditiva en [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citado de [este blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matriz de atención $\{\alpha_{i,j}\}$ representaría el grado en que ciertas palabras de entrada influyen en la generación de una palabra dada en la secuencia de salida. A continuación se muestra un ejemplo de dicha matriz:

![Imagen que muestra una alineación de muestra encontrada por RNNsearch-50, tomada de Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Figura tomada de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Los mecanismos de atención son responsables de gran parte del estado del arte actual o cercano al actual en el procesamiento de lenguaje natural. Sin embargo, agregar atención aumenta significativamente el número de parámetros del modelo, lo que llevó a problemas de escalabilidad con las RNNs. Una restricción clave para escalar las RNNs es que la naturaleza recurrente de los modelos dificulta el procesamiento por lotes y la paralelización del entrenamiento. En una RNN, cada elemento de una secuencia debe procesarse en orden secuencial, lo que significa que no se puede paralelizar fácilmente.

La adopción de mecanismos de atención combinada con esta restricción llevó a la creación de los modelos transformadores, ahora estado del arte, que conocemos y usamos hoy en día, desde BERT hasta OpenGPT3.

## Modelos transformadores

En lugar de transmitir el contexto de cada predicción anterior al siguiente paso de evaluación, los **modelos transformadores** utilizan **codificaciones posicionales** y **atención** para capturar el contexto de una entrada dada dentro de una ventana de texto proporcionada. La imagen a continuación muestra cómo las codificaciones posicionales con atención pueden capturar el contexto dentro de una ventana dada.

![GIF animado que muestra cómo se realizan las evaluaciones en los modelos transformadores.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Dado que cada posición de entrada se mapea de manera independiente a cada posición de salida, los transformadores pueden paralelizar mejor que las RNNs, lo que permite modelos de lenguaje mucho más grandes y expresivos. Cada cabeza de atención puede usarse para aprender diferentes relaciones entre palabras que mejoran las tareas de procesamiento de lenguaje natural.

## Construcción de un modelo transformador simple

Keras no contiene una capa de transformador integrada, pero podemos construir la nuestra. Como antes, nos enfocaremos en la clasificación de texto del conjunto de datos AG News, aunque vale la pena mencionar que los modelos transformadores muestran los mejores resultados en tareas de procesamiento de lenguaje natural más complejas.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Las nuevas capas en Keras deben ser subclases de la clase `Layer` e implementar el método `call`. Comencemos con la capa **Positional Embedding**. Usaremos [algo de código de la documentación oficial de Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Supondremos que rellenamos todas las secuencias de entrada hasta la longitud `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Esta capa consta de dos capas `Embedding`: una para incrustar tokens (de la manera que hemos discutido antes) y otra para las posiciones de los tokens. Las posiciones de los tokens se crean como una secuencia de números naturales desde 0 hasta `maxlen` utilizando `tf.range`, y luego se pasan a través de la capa de incrustación. Los dos vectores de incrustación resultantes se suman, produciendo una representación incrustada posicionalmente de la entrada con forma `maxlen`$\times$`embed_dim`.

Ahora, implementemos el bloque transformer. Este tomará la salida de la capa de incrustación definida anteriormente:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

El Transformer aplica `MultiHeadAttention` al input codificado posicionalmente para producir el vector de atención con dimensiones `maxlen`$\times$`embed_dim`, que luego se mezcla con el input y se normaliza utilizando `LayerNormalization`.

> **Nota**: `LayerNormalization` es similar a `BatchNormalization`, discutido en la parte de *Visión por Computadora* de este camino de aprendizaje, pero normaliza las salidas de la capa anterior para cada muestra de entrenamiento de manera independiente, llevándolas al rango [-1..1].

La salida de esta capa se pasa luego a través de una red `Dense` (en nuestro caso, un perceptrón de dos capas), y el resultado se suma a la salida final (que nuevamente se somete a normalización). 

Ahora, estamos listos para definir el modelo completo del Transformer:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modelos Transformadores BERT

**BERT** (Representaciones de Codificadores Bidireccionales de Transformadores) es una red de transformadores de múltiples capas muy grande, con 12 capas para *BERT-base* y 24 para *BERT-large*. El modelo se preentrena inicialmente en un gran corpus de datos de texto (Wikipedia + libros) utilizando entrenamiento no supervisado (predicción de palabras enmascaradas en una oración). Durante el preentrenamiento, el modelo adquiere un nivel significativo de comprensión del lenguaje que luego puede aprovecharse con otros conjuntos de datos mediante ajuste fino. Este proceso se llama **aprendizaje por transferencia**.

![imagen de http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Existen muchas variaciones de arquitecturas de Transformadores, incluyendo BERT, DistilBERT, BigBird, OpenGPT3 y más, que pueden ajustarse finamente.

Veamos cómo podemos usar un modelo BERT preentrenado para resolver nuestro problema tradicional de clasificación de secuencias. Tomaremos la idea y algo de código de la [documentación oficial](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Para cargar modelos preentrenados, utilizaremos **Tensorflow hub**. Primero, carguemos el vectorizador específico de BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Es importante que utilices el mismo vectorizador que se usó para entrenar la red original. Además, el vectorizador de BERT devuelve tres componentes:
* `input_word_ids`, que es una secuencia de números de tokens para la oración de entrada
* `input_mask`, que muestra qué parte de la secuencia contiene la entrada real y cuál es relleno. Es similar a la máscara producida por la capa `Masking`
* `input_type_ids` se utiliza para tareas de modelado de lenguaje y permite especificar dos oraciones de entrada en una sola secuencia.

Luego, podemos instanciar el extractor de características de BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Entonces, la capa BERT devuelve una serie de resultados útiles:
* `pooled_output` es el resultado de promediar todos los tokens en la secuencia. Puedes verlo como una representación semántica inteligente de toda la red. Es equivalente a la salida de la capa `GlobalAveragePooling1D` en nuestro modelo anterior.
* `sequence_output` es la salida de la última capa del transformador (corresponde a la salida de `TransformerBlock` en nuestro modelo anterior).
* `encoder_outputs` son las salidas de todas las capas del transformador. Dado que hemos cargado un modelo BERT de 4 capas (como probablemente puedas deducir del nombre, que contiene `4_H`), tiene 4 tensores. El último es el mismo que `sequence_output`.

Ahora definiremos el modelo de clasificación de extremo a extremo. Usaremos la *definición funcional del modelo*, donde definimos la entrada del modelo y luego proporcionamos una serie de expresiones para calcular su salida. También haremos que los pesos del modelo BERT no sean entrenables y entrenaremos solo el clasificador final:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


A pesar de que hay pocos parámetros entrenables, el proceso es bastante lento porque el extractor de características de BERT es computacionalmente pesado. Parece que no pudimos alcanzar una precisión razonable, ya sea por falta de entrenamiento o por falta de parámetros en el modelo.

Intentemos desbloquear los pesos de BERT y entrenarlo también. Esto requiere una tasa de aprendizaje muy pequeña y una estrategia de entrenamiento más cuidadosa con **warmup**, utilizando el optimizador **AdamW**. Usaremos el paquete `tf-models-official` para crear el optimizador:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Como puedes ver, el entrenamiento avanza bastante lento, pero podrías experimentar y entrenar el modelo durante algunas épocas (5-10) para ver si puedes obtener el mejor resultado en comparación con los enfoques que hemos utilizado anteriormente.

## Biblioteca Huggingface Transformers

Otra forma muy común (y un poco más sencilla) de usar modelos Transformer es el [paquete HuggingFace](https://github.com/huggingface/), que proporciona bloques simples para diferentes tareas de PLN. Está disponible tanto para Tensorflow como para PyTorch, otro marco de redes neuronales muy popular.

> **Nota**: Si no estás interesado en ver cómo funciona la biblioteca Transformers, puedes saltar al final de este notebook, ya que no verás nada sustancialmente diferente de lo que hemos hecho anteriormente. Estaremos repitiendo los mismos pasos de entrenamiento del modelo BERT utilizando una biblioteca diferente y un modelo sustancialmente más grande. Por lo tanto, el proceso implica un entrenamiento bastante largo, así que podrías simplemente revisar el código.

Veamos cómo se puede resolver nuestro problema utilizando [Huggingface Transformers](http://huggingface.co).


Lo primero que necesitamos hacer es elegir el modelo que vamos a utilizar. Además de algunos modelos integrados, Huggingface cuenta con un [repositorio de modelos en línea](https://huggingface.co/models), donde puedes encontrar muchos más modelos preentrenados creados por la comunidad. Todos esos modelos se pueden cargar y usar simplemente proporcionando el nombre del modelo. Todos los archivos binarios necesarios para el modelo se descargarán automáticamente.

En ciertos casos, necesitarás cargar tus propios modelos, en cuyo caso puedes especificar el directorio que contiene todos los archivos relevantes, incluyendo los parámetros para el tokenizer, el archivo `config.json` con los parámetros del modelo, los pesos binarios, etc.

A partir del nombre del modelo, podemos instanciar tanto el modelo como el tokenizer. Comencemos con un tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

El objeto `tokenizer` contiene la función `encode` que puede usarse directamente para codificar texto:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Podemos también usar el tokenizador para codificar una secuencia de una manera adecuada para pasar al modelo, es decir, incluyendo los campos `token_ids`, `input_mask`, etc. También podemos especificar que queremos tensores de Tensorflow proporcionando el argumento `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

En nuestro caso, utilizaremos un modelo BERT preentrenado llamado `bert-base-uncased`. *Uncased* indica que el modelo no distingue entre mayúsculas y minúsculas.

Al entrenar el modelo, necesitamos proporcionar una secuencia tokenizada como entrada, y por lo tanto diseñaremos una canalización de procesamiento de datos. Dado que `tokenizer.encode` es una función de Python, utilizaremos el mismo enfoque que en la última unidad llamándola mediante `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Ahora podemos cargar el modelo real utilizando el paquete `BertForSequenceClassification`. Esto asegura que nuestro modelo ya tenga una arquitectura requerida para la clasificación, incluyendo el clasificador final. Verás un mensaje de advertencia que indica que los pesos del clasificador final no están inicializados y que el modelo requeriría preentrenamiento; eso está perfectamente bien, porque es exactamente lo que estamos a punto de hacer.


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Como puedes ver en `summary()`, ¡el modelo contiene casi 110 millones de parámetros! Presumiblemente, si queremos una tarea de clasificación simple en un conjunto de datos relativamente pequeño, no queremos entrenar la capa base de BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


¡Ahora estamos listos para comenzar el entrenamiento!

> **Nota**: ¡Entrenar un modelo BERT a gran escala puede llevar mucho tiempo! Por lo tanto, solo lo entrenaremos durante las primeras 32 tandas. Esto es solo para mostrar cómo se configura el entrenamiento del modelo. Si estás interesado en probar un entrenamiento a gran escala, simplemente elimina los parámetros `steps_per_epoch` y `validation_steps`, ¡y prepárate para esperar!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Si aumentas el número de iteraciones y esperas lo suficiente, y entrenas durante varias épocas, puedes esperar que la clasificación con BERT nos dé la mejor precisión. Esto se debe a que BERT ya comprende bastante bien la estructura del lenguaje, y solo necesitamos ajustar el clasificador final. Sin embargo, dado que BERT es un modelo grande, todo el proceso de entrenamiento lleva mucho tiempo y requiere una potencia computacional considerable (GPU, y preferiblemente más de una).

> **Note:** En nuestro ejemplo, hemos estado utilizando uno de los modelos BERT preentrenados más pequeños. Existen modelos más grandes que probablemente ofrezcan mejores resultados.


## Puntos clave

En esta unidad, hemos explorado arquitecturas de modelos muy recientes basadas en **transformers**. Las hemos aplicado a nuestra tarea de clasificación de texto, pero de manera similar, los modelos BERT pueden utilizarse para extracción de entidades, respuesta a preguntas y otras tareas de procesamiento de lenguaje natural.

Los modelos basados en transformers representan el estado del arte actual en NLP, y en la mayoría de los casos deberían ser la primera solución con la que empieces a experimentar al implementar soluciones personalizadas de NLP. Sin embargo, comprender los principios básicos subyacentes de las redes neuronales recurrentes discutidos en este módulo es extremadamente importante si deseas construir modelos neuronales avanzados.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
